In [4]:
import pandas as pd
filepath = 'train.tsv'
filepath1 = 'dev_part1.tsv'
df = pd.read_csv(filepath, sep='\t', encoding = "ISO-8859-1")
wf = pd.read_csv(filepath1, sep='\t', encoding = "ISO-8859-1")

df.head() 

,PMID,Sentence_ID,Sentence,Gene1|Gene1_ID,Gene1_Index(start|end),Gene2|Gene2_ID,Gene2_Index(start|end),RE_Type
0,24019522,S7,"Notably, all of the H3K36-specific methyltrans...",ASH1L|55870,65|70,HYPB|29072,72|76,NoRE
1,24019522,S7,"Notably, all of the H3K36-specific methyltrans...",ASH1L|55870,65|70,NSD1|64324,78|82,NoRE
2,24019522,S7,"Notably, all of the H3K36-specific methyltrans...",ASH1L|55870,65|70,NSD2|7468,88|92,NoRE
3,24019522,S7,"Notably, all of the H3K36-specific methyltrans...",ASH1L|55870,65|70,Pr-Set7|387893,189|196,NoRE
4,24019522,S7,"Notably, all of the H3K36-specific methyltrans...",ASH1L|55870,65|70,H2A|8337,113|116,Negative_Regulation


In [5]:
sentences = df['Sentence']
target = df['RE_Type']

In [6]:
sentences.head()

0    Notably, all of the H3K36-specific methyltrans...
1    Notably, all of the H3K36-specific methyltrans...
2    Notably, all of the H3K36-specific methyltrans...
3    Notably, all of the H3K36-specific methyltrans...
4    Notably, all of the H3K36-specific methyltrans...
Name: Sentence, dtype: object

In [7]:
target.head()

0                   NoRE
1                   NoRE
2                   NoRE
3                   NoRE
4    Negative_Regulation
Name: RE_Type, dtype: object

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df = 1)
vectorizer.fit(sentences)
vectorizer.vocabulary_

{'notably': 8901,
 'all': 999,
 'of': 9068,
 'the': 12717,
 'h3k36': 5736,
 'specific': 12005,
 'methyltransferases': 8006,
 'including': 6560,
 'ash1l': 1395,
 'hypb': 6200,
 'nsd1': 8954,
 'and': 1138,
 'nsd2': 8955,
 'were': 13700,
 'inhibited': 6666,
 'by': 2018,
 'ubh2a': 13247,
 'whereas': 13713,
 'other': 9245,
 'histone': 6004,
 'prc2': 10136,
 'g9a': 5333,
 'pr': 10130,
 'set7': 11648,
 'not': 8898,
 'affected': 878,
 'three': 12765,
 'tetratricopeptide': 12679,
 'repeat': 10955,
 'domain': 4082,
 'containing': 3047,
 'docking': 4072,
 'proteins': 10404,
 'toc64': 12878,
 'om64': 9101,
 'attpr7': 1523,
 'reside': 11028,
 'in': 6537,
 'chloroplast': 2575,
 'mitochondrion': 8160,
 'endoplasmic': 4487,
 'reticulum': 11095,
 'arabidopsis': 1309,
 'thaliana': 12712,
 'respectively': 11047,
 'here': 5915,
 'we': 13688,
 'performed': 9647,
 'detailed': 3787,
 'biochemical': 1797,
 'biophysical': 1812,
 'computational': 2934,
 'analysis': 1124,
 'interaction': 6770,
 'between': 1755,


In [9]:
voc_dic=vectorizer.vocabulary_
len(voc_dic) #共有16150個unique words

13905

In [10]:
vectorizer.transform(sentences).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
sentences = df['Sentence']
target = df['RE_Type']
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, target,
test_size=0.25, random_state=1000) #將data根據 1:3切成 test 和 train
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)
# train 和 test 分別做出feature vector (BOW)
X_train = vectorizer.transform(sentences_train)
X_test = vectorizer.transform(sentences_test)
classifier = LogisticRegression() #初始化 LR model
classifier.fit(X_train, y_train) # 訓練 LR model
score = classifier.score(X_test, y_test) #評估 LR model

score

0.7979723773141346

In [12]:
sentences1 = df['Sentence']
sentences2 = wf['Sentence']
target = df['RE_Type']
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
vectorizer = CountVectorizer()
vectorizer.fit(sentences1)
# train 和 test 分別做出feature vector (BOW)
X_train = vectorizer.transform(sentences1)
X_test = vectorizer.transform(sentences2)
y_train = target
classifier = LogisticRegression() #初始化 LR model
classifier.fit(X_train, y_train) # 訓練 LR model
result = classifier.predict(X_test)

寫入檔案

In [13]:
pre = pd.DataFrame({'RE_Type':result})
wf = pd.concat([wf, pre], sort = False, axis = 1)
wf.to_csv("prediction_simple.tsv", encoding = 'utf-8', sep = '\t', index = False)